# Application of Eigenvector Centrality in Ranking Fan-Favorite Characters from Love Live! Nijigasaki High School Idol Club

Study by Muhammad Ghifary Komara Putra (13523066)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("nijigaku_popularity_poll.xlsx")
df = df.drop(columns=["No."])
df.head()

,Ayumu Uehara,Kasumi Nakasu,Setsuna Yuki,Ai Miyashita,Emma Verde,Rina Tennoji,Kanata Konoe,Shizuku Osaka,Karin Asaka
0,4,3,2,9,7,1,8,6,5
1,6,7,1,9,5,2,4,3,8
2,6,4,1,7,8,2,9,3,5
3,9,4,2,7,8,3,6,5,1
4,8,4,1,5,9,7,3,6,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Ayumu Uehara   30 non-null     int64
 1   Kasumi Nakasu  30 non-null     int64
 2   Setsuna Yuki   30 non-null     int64
 3   Ai Miyashita   30 non-null     int64
 4   Emma Verde     30 non-null     int64
 5   Rina Tennoji   30 non-null     int64
 6   Kanata Konoe   30 non-null     int64
 7   Shizuku Osaka  30 non-null     int64
 8   Karin Asaka    30 non-null     int64
dtypes: int64(9)
memory usage: 2.2 KB


In [4]:
characters = df.columns
characters

Index(['Ayumu Uehara', 'Kasumi Nakasu', 'Setsuna Yuki', 'Ai Miyashita',
       'Emma Verde', 'Rina Tennoji', 'Kanata Konoe', 'Shizuku Osaka',
       'Karin Asaka'],
      dtype='object')

### 1. Converting Rank to Score

In [5]:
score_df = 9 - (df - 1)
score_df.head()

,Ayumu Uehara,Kasumi Nakasu,Setsuna Yuki,Ai Miyashita,Emma Verde,Rina Tennoji,Kanata Konoe,Shizuku Osaka,Karin Asaka
0,6,7,8,1,3,9,2,4,5
1,4,3,9,1,5,8,6,7,2
2,4,6,9,3,2,8,1,7,5
3,1,6,8,3,2,7,4,5,9
4,2,6,9,5,1,3,7,4,8


### 2. Converting Score to Adjacency Matrix

In [6]:
# borda count
score_arr = score_df.sum(axis=0).to_numpy()
print(score_arr)

[144 161 236 121  90 116 150 125 207]


In [7]:
# Variation 1

A1 = np.zeros((9,9))
score_matrix = score_df.to_numpy()

for i in range(30):
    for j in range(9):
        for k in range(9):
            if score_matrix[i][j] > score_matrix[i][k]:
                A1[j][k] += 1
        
print(A1)

[[ 0. 12.  4. 16. 23. 19. 12. 21.  7.]
 [18.  0.  3. 21. 24. 20. 17. 21.  7.]
 [26. 27.  0. 29. 28. 25. 25. 26. 20.]
 [14.  9.  1.  0. 21. 18. 11. 14.  3.]
 [ 7.  6.  2.  9.  0. 14.  9. 10.  3.]
 [11. 10.  5. 12. 16.  0. 12. 12.  8.]
 [18. 13.  5. 19. 21. 18.  0. 17.  9.]
 [ 9.  9.  4. 16. 20. 18. 13.  0.  6.]
 [23. 23. 10. 27. 27. 22. 21. 24.  0.]]


In [8]:
# Variation 2

A2 = np.zeros((9,9))

for i in range(9):
    for j in range(9):
        difference = score_arr[i] - score_arr[j]
        if difference > 0:
            A2[i][j] = difference
            
print(A2)

[[  0.   0.   0.  23.  54.  28.   0.  19.   0.]
 [ 17.   0.   0.  40.  71.  45.  11.  36.   0.]
 [ 92.  75.   0. 115. 146. 120.  86. 111.  29.]
 [  0.   0.   0.   0.  31.   5.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  26.   0.   0.   0.   0.]
 [  6.   0.   0.  29.  60.  34.   0.  25.   0.]
 [  0.   0.   0.   4.  35.   9.   0.   0.   0.]
 [ 63.  46.   0.  86. 117.  91.  57.  82.   0.]]


### 3. Computing Eigenvalues and Eigenvectors

In [9]:
# Variation 1

eigenvalues1, eigenvectors1 = np.linalg.eig(A1)
print(eigenvalues1)

[104.24140207+0.j          -8.06011376+6.49130734j
  -8.06011376-6.49130734j -14.57493881+4.51750859j
 -14.57493881-4.51750859j -14.89532883+1.91450793j
 -14.89532883-1.91450793j -14.59031963+0.25919106j
 -14.59031963-0.25919106j]


In [10]:
norm = np.linalg.norm(eigenvectors1[:,0])
result1 = eigenvectors1[:,0] / norm
result1 = np.abs(result1.real)
print(result1)

[0.29056116 0.3275653  0.54762684 0.2267826  0.16476132 0.23975175
 0.31344951 0.24783443 0.46113988]


In [11]:
# Variation 2

epsilon = 0.00001
A2 += epsilon

eigenvalues2, eigenvectors2 = np.linalg.eig(A2)
print(eigenvalues2)

[ 4.5025866 +0.j          2.13526349+3.39751503j  2.13526349-3.39751503j
 -0.91207661+2.92168156j -0.91207661-2.92168156j -1.87305654+1.23246452j
 -1.87305654-1.23246452j -1.47134414+0.j         -1.73141315+0.j        ]


In [12]:
norm = np.linalg.norm(eigenvectors2[:,0])
result2 = eigenvectors2[:,0] / norm
result2 = np.abs(result2.real)
print(result2)

[7.15337495e-04 8.53430139e-03 9.92292247e-01 3.85714287e-05
 2.50337296e-06 1.69589965e-05 1.86600767e-03 9.01273928e-05
 1.23609302e-01]


## Results

In [13]:
# Borda Count

normalized_score_arr = score_arr/np.linalg.norm(score_arr)
result_borda_count_df = pd.DataFrame([np.round(normalized_score_arr,3)], columns=characters)
sorted_result_borda_count_df = result_borda_count_df.loc[:,result_borda_count_df.iloc[0].sort_values(ascending=False).index]
sorted_result_borda_count_df

,Setsuna Yuki,Karin Asaka,Kasumi Nakasu,Kanata Konoe,Ayumu Uehara,Shizuku Osaka,Ai Miyashita,Rina Tennoji,Emma Verde
0,0.504,0.442,0.344,0.32,0.307,0.267,0.258,0.248,0.192


In [14]:
# variation 1

result1_df = pd.DataFrame([np.round(result1,3)], columns=characters)

sorted_result1_df = result1_df.loc[:,result1_df.iloc[0].sort_values(ascending=False).index]
sorted_result1_df

,Setsuna Yuki,Karin Asaka,Kasumi Nakasu,Kanata Konoe,Ayumu Uehara,Shizuku Osaka,Rina Tennoji,Ai Miyashita,Emma Verde
0,0.548,0.461,0.328,0.313,0.291,0.248,0.24,0.227,0.165


In [15]:
# variation 2

result2_df = pd.DataFrame([np.round(result2,6)], columns=characters)

sorted_result2_df = result2_df.loc[:,result2_df.iloc[0].sort_values(ascending=False).index]
sorted_result2_df

,Setsuna Yuki,Karin Asaka,Kasumi Nakasu,Kanata Konoe,Ayumu Uehara,Shizuku Osaka,Ai Miyashita,Rina Tennoji,Emma Verde
0,0.992292,0.123609,0.008534,0.001866,0.000715,0.00009,0.000039,0.000017,0.000003


### Further Statistics

In [16]:
normalized_score_arr.max() - normalized_score_arr.min()

0.3116103341453932

In [17]:
result1.max() - result1.min()

0.38286552197400225

In [18]:
result2.max() - result2.min()

0.9922897431627059

In [19]:
from scipy.stats import skew

In [20]:
skew(normalized_score_arr)

0.711831758807111

In [21]:
skew(result1)

0.864647852863504

In [22]:
skew(result2)

2.4069911932831864